In [25]:
import os
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.tools import Tool
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain import LLMMathChain

import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

from typing import List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.tools import Tool, StructuredTool
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.schema import Document

load_dotenv()

True

In [26]:
import os
from typing import List
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.schema import Document

class PatientRAG:
    def __init__(self, patients_directory: str):
        self.patients_directory = patients_directory
        self.embeddings = OpenAIEmbeddings()
        self.vector_store = self.load_patient_data()

    def load_patient_data(self):
        documents = []
        for filename in os.listdir(self.patients_directory):
            if filename.endswith('.txt'):
                file_path = os.path.join(self.patients_directory, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    documents.append(Document(page_content=content, metadata={"source": filename}))
        
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(documents)
        return Chroma.from_documents(splits, self.embeddings)

    def query(self, question):
        if isinstance(question, dict) and 'args' in question:
            question = ' '.join(question['args'])
        elif isinstance(question, list):
            question = ' '.join(question)
        
        # Primero, intentamos buscar por nombre de archivo
        words = question.lower().split()
        patient_name = words[-1] if words else ""  # Asumimos que el nombre del paciente es la última palabra
        file_name = f"{patient_name}.txt"
        file_path = os.path.join(self.patients_directory, file_name)
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                return f"Información del paciente {patient_name.capitalize()}:\n{file.read()}"
        
        # Si no se encuentra por nombre de archivo, usamos el retriever
        retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})
        qa_chain = RetrievalQA.from_chain_type(
            llm=ChatOpenAI(temperature=0, model_name="gpt-4"),
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )
        result = qa_chain({"query": question})
        
        if not result['result']:
            return f"No se encontró información para la consulta: {question}"
        
        return f"Respuesta: {result['result']}\nFuentes: {[doc.metadata['source'] for doc in result['source_documents']]}"

In [27]:
class DoctorAgent:
    def diagnose(self, symptoms):
        if isinstance(symptoms, list):
            symptoms = ' '.join(symptoms)
        return f"Basado en los síntomas '{symptoms}', el diagnóstico preliminar es..."

class NurseAgent:
    def check_vitals(self, patient_name):
        if isinstance(patient_name, list):
            patient_name = patient_name[0]
        return f"Los signos vitales de {patient_name} son normales."

class AdminAgent:
    def schedule_appointment(self, input_data):
        if isinstance(input_data, list):
            input_string = ' '.join(input_data)
        else:
            input_string = input_data
        parts = input_string.split(maxsplit=1)
        if len(parts) != 2:
            return "Error: Por favor, proporcione el nombre del paciente y la fecha separados por un espacio."
        patient_name, date = parts
        return f"Cita agendada para {patient_name} el {date}."

In [28]:
from langchain.tools import Tool

class HospitalTools:
    def __init__(self, rag_system):
        self.rag_system = rag_system
        self.doctor = DoctorAgent()
        self.nurse = NurseAgent()
        self.admin = AdminAgent()

    def list_patients(self):
        return ', '.join([f.split('.')[0] for f in os.listdir(self.rag_system.patients_directory) if f.endswith('.txt')])

    def get_tools(self):
        return [
            Tool(
                name="consulta_paciente",
                func=self.rag_system.query,
                description="Útil para buscar información específica sobre un paciente."
            ),
            Tool(
                name="diagnostico",
                func=self.doctor.diagnose,
                description="Útil para obtener un diagnóstico preliminar basado en síntomas."
            ),
            Tool(
                name="verificar_signos_vitales",
                func=self.nurse.check_vitals,
                description="Útil para verificar los signos vitales de un paciente."
            ),
            Tool(
                name="agendar_cita",
                func=self.admin.schedule_appointment,
                description="Útil para agendar una cita para un paciente."
            ),
            Tool(
                name="listar_pacientes",
                func=self.list_patients,
                description="Útil para obtener una lista de todos los pacientes disponibles."
            )
        ]

In [29]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor

class HospitalAgent:
    def __init__(self, tools):
        self.llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o-mini")
        prompt = hub.pull("hwchase17/openai-functions-agent")
        self.agent = create_openai_functions_agent(self.llm, tools, prompt)
        self.agent_executor = AgentExecutor(agent=self.agent, tools=tools, verbose=True)

    def run(self, query: str) -> Dict[str, str]:
        result = self.agent_executor.invoke({"input": query})
        tool_used = result.get('intermediate_steps', [])
        if tool_used:
            tool_name = tool_used[0][0].tool
        else:
            tool_name = "No se utilizó ninguna herramienta específica"
        return {
            "output": result["output"],
            "tool_used": tool_name
        }

In [30]:
import logging

logging.basicConfig(level=logging.INFO)

import logging
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor

class HospitalApplication:
    def __init__(self, patients_directory: str):
        self.patients_directory = patients_directory
        rag_system = PatientRAG(patients_directory)
        hospital_tools = HospitalTools(rag_system)
        
        llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
        prompt = hub.pull("hwchase17/openai-functions-agent")
        tools = hospital_tools.get_tools()
        agent = create_openai_functions_agent(llm, tools, prompt)
        self.agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    def process_query(self, query: str):
        try:
            logging.info(f"Consulta recibida: {query}")
            response = self.agent_executor.invoke({"input": query})
            output = response['output']
            if 'intermediate_steps' in response:
                tool_used = response['intermediate_steps'][0][0].tool if response['intermediate_steps'] else "Ninguna herramienta utilizada"
            else:
                tool_used = "Información no disponible"
            
            logging.info(f"Respuesta generada: {output}")
            print("\nRespuesta:", output)
            print(f"Herramienta utilizada: {tool_used}")
            
            return output, tool_used
        except Exception as e:
            logging.error(f"Error ocurrido: {str(e)}")
            error_message = f"Ocurrió un error al procesar la consulta: {str(e)}"
            print("\n" + error_message)
            return error_message, "Error"

In [31]:
import logging
from dotenv import load_dotenv

# Configuración inicial
load_dotenv()
logging.basicConfig(level=logging.INFO)

# Directorio de datos de pacientes
PATIENTS_DIRECTORY = "../data/pacientes"

# Inicializar la aplicación del hospital
app = HospitalApplication(PATIENTS_DIRECTORY)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [32]:
import os

def create_sample_patient_files(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    patients = {
        "juan.txt": "Nombre: Juan Pérez\nEdad: 35\nHistorial: Hipertensión controlada",
        "maria.txt": "Nombre: María González\nEdad: 28\nHistorial: Sin condiciones previas",
        "pedro.txt": "Nombre: Pedro Rodríguez\nEdad: 45\nHistorial: Diabetes tipo 2",
        "ana.txt": "Nombre: Ana Martínez\nEdad: 52\nHistorial: Artritis reumatoide"
    }
    
    for filename, content in patients.items():
        with open(os.path.join(directory, filename), 'w') as f:
            f.write(content)

create_sample_patient_files("../data/pacientes")

In [33]:
# Ejemplos de consultas para probar
queries = [
    "¿Cuáles son los pacientes disponibles?",
    "¿Cuál es la información del paciente Juan?",
    "Diagnostica dolor de cabeza y fiebre",
    "Agenda una cita para María el 15 de agosto",
    "¿Cuáles son los signos vitales de Pedro?",
    "Busca información sobre el historial médico de Ana"
]

# Procesar cada consulta
for query in queries:
    print(f"\n--- Procesando: {query} ---")
    output, tool = app.process_query(query)

print("\n¡Gracias por usar el Sistema de Consulta de Pacientes del Hospital!")

INFO:root:Consulta recibida: ¿Cuáles son los pacientes disponibles?



--- Procesando: ¿Cuáles son los pacientes disponibles? ---


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
ERROR:root:Error ocurrido: Too many arguments to single-input tool listar_pacientes.
                Consider using StructuredTool instead. Args: []
INFO:root:Consulta recibida: ¿Cuál es la información del paciente Juan?



Invoking: `listar_pacientes` with `{}`



Ocurrió un error al procesar la consulta: Too many arguments to single-input tool listar_pacientes.
                Consider using StructuredTool instead. Args: []

--- Procesando: ¿Cuál es la información del paciente Juan? ---


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Los signos vitales de Pedro son normales.

> Finished chain.

Respuesta: Los signos vitales de Pedro son normales.
Herramienta utilizada: Información no disponible

--- Procesando: Busca información sobre el historial médico de Ana ---


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `consulta_paciente` with `{'args': ['Ana']}`


Información del paciente Ana:
Nombre: Ana Martínez
Edad: 52
Historial: Artritis reumatoide

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Respuesta generada: La información del paciente Ana Martínez es la siguiente:
- Edad: 52 años
- Historial médico: Se le ha diagnosticado con artritis reumatoide.


La información del paciente Ana Martínez es la siguiente:
- Edad: 52 años
- Historial médico: Se le ha diagnosticado con artritis reumatoide.

> Finished chain.

Respuesta: La información del paciente Ana Martínez es la siguiente:
- Edad: 52 años
- Historial médico: Se le ha diagnosticado con artritis reumatoide.
Herramienta utilizada: Información no disponible

¡Gracias por usar el Sistema de Consulta de Pacientes del Hospital!
